In [1]:
!pwd
# ! unzip data.zip
! git clone https://github.com/chian-chen/BELab.git

/content
Cloning into 'BELab'...
remote: Enumerating objects: 808, done.
remote: Counting objects: 100% (808/808), done.
remote: Compressing objects: 100% (804/804), done.
remote: Total 808 (delta 24), reused 773 (delta 3), pack-reused 0
Receiving objects: 100% (808/808), 1.79 MiB | 7.97 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
import numpy as np
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from scipy import signal

In [26]:
def normalize(signal):
    for s in signal:
        # s = (s-np.mean(s))/np.sqrt(np.var(s))
        s = (s-np.mean(s))
    return signal
def find_max_segment_power(signal, length):
    max_i = 0
    max_pow = -1
    for i in range(signal.shape[0] - length):
        if np.square(signal[i:i+length]).sum() > max_pow:
            max_i = i
            max_pow = np.square(signal[i:i+length]).sum()
            # print(max_i, max_pow)
        # print(i, i+length, np.square(signal[i:i+length]).sum(),  max_pow)
    return signal[max_i:max_i+length]

def process_signal(signal):
    data = np.array([
        signal['x'], signal['rx'],  
        signal['y'], signal['ry'], signal['z'], signal['rz'],
    ])

    length = 80
    process_data = np.zeros((data.shape[0], length))

    for i in range(len(data)):
        data[i] = normalize(data[i])
        process_data[i] = find_max_segment_power(data[i], length)
        # print(s.shape)
    # print(process_data.shape)

    return process_data
    #return normalize(data)
def preprocess_data(file_path):
    X = []
    y = []
    num_of_bad_data = 0
    label2digit = {"GestureDown":0, "GestureLeft":1, "GestureN":2,
                   "GestureO": 3, "GestureRight":4, "GestureUp":5, 
                   "GestureV": 6, "GestureZ": 7, "Noise":8
                   }
    for folder in os.listdir(file_path):
        if os.path.isdir(f"{file_path}/{folder}"):
            for file in os.listdir(f"{file_path}/{folder}"):
                if os.path.isfile(f"{file_path}/{folder}/{file}"):
                    if file.split('.')[1] == 'npz' and file.split('.')[0].split('_')[0]!='hung':
                        signal = np.load(f"{file_path}/{folder}/{file}")
                        is_good_data = True
                        for rx_data in signal['rx']:
                            if rx_data > 1000 or rx_data < -1000:
                                print(f"{file_path}/{folder}/{file}")
                                num_of_bad_data += 1
                                is_good_data = False
                                break
                        if is_good_data:
                            signal = process_signal(signal)
                            # print(signal.shape)
                            if not np.isnan(signal).any() :
                                X.append(np.expand_dims(signal, 0).tolist())
                                y.append(label2digit[folder])
                            else:
                                num_of_bad_data += 1
                                print(f"{file_path}/{folder}/{file}")
    print(f'Total {num_of_bad_data} bad data!')
    # print(X)
    # return X, y
    return torch.tensor(X), torch.tensor(y)
X, y = preprocess_data('/content/BELab/data')

/content/BELab/data/GestureN/weng_3.npz
/content/BELab/data/GestureN/ko_7.npz
/content/BELab/data/GestureN/weng_2.npz
/content/BELab/data/GestureN/weng_5.npz
/content/BELab/data/GestureN/weng_1.npz
/content/BELab/data/GestureN/ko_3.npz
/content/BELab/data/GestureZ/yang_2.npz
/content/BELab/data/GestureZ/weng_7.npz
/content/BELab/data/GestureZ/weng_17.npz
/content/BELab/data/GestureZ/yang_10.npz
/content/BELab/data/GestureZ/weng_16.npz
/content/BELab/data/GestureZ/ko_8.npz
/content/BELab/data/GestureZ/ko_15.npz
/content/BELab/data/GestureZ/weng_13.npz
/content/BELab/data/GestureZ/weng_8.npz
/content/BELab/data/GestureZ/ko_1.npz
/content/BELab/data/GestureZ/ko_0.npz
/content/BELab/data/GestureV/weng_9.npz
/content/BELab/data/GestureV/ko_14.npz
/content/BELab/data/GestureV/ko_7.npz
/content/BELab/data/GestureV/ko_17.npz
/content/BELab/data/GestureV/ko_10.npz
/content/BELab/data/GestureV/ko_11.npz
/content/BELab/data/GestureV/ko_5.npz
/content/BELab/data/GestureV/weng_0.npz
/content/BELab/

In [27]:
# sig = np.load('data/GestureLeft/chen_0.npz')
# d = process_signal(sig)
# # print(d)

In [28]:
X[0].shape

torch.Size([1, 6, 80])

In [29]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, signals, labels):
        self.data = signals
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [30]:
batch_size = 10

my_data = MyDataset(X, y)

# Define the sizes of the training and validation sets
train_size = int(0.9 * len(my_data))
val_size = len(my_data) - train_size

# Split the dataset into training and validation sets
train_data, val_data = random_split(my_data, [train_size, val_size])

# Create data loaders for the training and validation sets
train_loader = DataLoader(train_data, batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size, shuffle=False)

In [36]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
# Define model
class CNN(nn.Module):
    def __init__(self, num_class, out_channel=196):
        super().__init__()
        self.num_class = num_class
        self.out_channel = out_channel
        self.cnn = nn.Conv2d(1, self.out_channel, (1, 16))
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d((2, 4), stride=(2, 4))
        self.fc = nn.Linear(self.out_channel*3*16, 256)
        self.dropout = nn.Dropout(0.5)
        self.proj_class = nn.Linear(256, self.num_class)


    def forward(self, x):
        x = self.cnn(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        x = self.dropout(x)
        x = self.proj_class(x)
        return x

model = CNN(9, 16).to(device)
print(model)

Using cuda device
CNN(
  (cnn): Conv2d(1, 16, kernel_size=(1, 16), stride=(1, 1))
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=(2, 4), stride=(2, 4), padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=768, out_features=256, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (proj_class): Linear(in_features=256, out_features=9, bias=True)
)


In [37]:
input = torch.randn(1, 1,6,80).to(device)
input2 = torch.randn(1, 1,6,80).to(device)
output = model(input2)
print(output.shape)

torch.Size([1, 9])


In [38]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)



In [39]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(size)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # print(X)
        # Compute prediction error
        pred = model(X)
        # print(pred)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [40]:
def valid(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # print("pred")
            # print(pred)
            # print('y')
            # print(y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    # print(size)
    # print(acc)
    acc = correct / size
    print(f"Test Error: \n Accuracy: {(100*acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return acc

In [41]:
epochs = 20
best_acc = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    cur_acc = valid(val_loader, model, loss_fn)
    if cur_acc > best_acc:
        best_acc = cur_acc
        print(f"Save better model on epoch {t}")
        torch.save(model.state_dict(), "best_model.pth")
print("Done!")

Epoch 1
-------------------------------
493
loss: 22.909239  [   10/  493]
loss: 22.453039  [  110/  493]
loss: 10.512362  [  210/  493]
loss: 17.908527  [  310/  493]
loss: 10.620424  [  410/  493]
Test Error: 
 Accuracy: 47.3%, Avg loss: 6.138021 

Save better model on epoch 0
Epoch 2
-------------------------------
493
loss: 7.518157  [   10/  493]
loss: 9.200394  [  110/  493]
loss: 9.641466  [  210/  493]
loss: 8.081459  [  310/  493]
loss: 17.994131  [  410/  493]
Test Error: 
 Accuracy: 49.1%, Avg loss: 3.427858 

Save better model on epoch 1
Epoch 3
-------------------------------
493
loss: 8.851155  [   10/  493]
loss: 6.429410  [  110/  493]
loss: 7.345438  [  210/  493]
loss: 7.756417  [  310/  493]
loss: 4.374694  [  410/  493]
Test Error: 
 Accuracy: 58.2%, Avg loss: 3.590443 

Save better model on epoch 2
Epoch 4
-------------------------------
493
loss: 5.640553  [   10/  493]
loss: 10.224886  [  110/  493]
loss: 7.756546  [  210/  493]
loss: 0.470666  [  310/  493]
loss

In [14]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
# clf = SVC(kernel='linear')
print(len(my_data))
pca = PCA(n_components=4)
pca.fit(my_data.data.view(len(my_data), -1))
signal_train = pca.transform(my_data.data.view(len(my_data), -1))

X_train, X_test, y_train, y_test = train_test_split(
    signal_train, my_data.labels, test_size=5/80, random_state=42)


clf.fit(X_train, y_train)
print(clf.predict(X_test))
print(y_test)

a = np.arange(10)
b = np.arange(10)
print(a == b)


542
[4 7 4 4 3 1 0 0 8 5 2 8 8 7 0 8 0 8 8 2 8 4 4 1 8 2 8 8 0 0 0 0 3 0]
tensor([1, 7, 1, 7, 0, 7, 0, 5, 5, 5, 2, 0, 8, 7, 5, 0, 5, 7, 5, 2, 6, 1, 7, 1,
        5, 2, 6, 8, 2, 6, 0, 6, 3, 3])
[ True  True  True  True  True  True  True  True  True  True]


In [15]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
iris = datasets.load_iris()
X = iris.data
y = iris.target
print(X.shape)
print(X[0])

(150, 4)
[5.1 3.5 1.4 0.2]
